In [ ]:
import pandas as pd
import os
from pathlib import Path

import matplotlib.pyplot as plt

#### Auxiliary functions used to generate plots

In [ ]:
def plot_energy_failed(competitors_pwr: dict[str, pd.DataFrame], 
                       competitors_failed : dict[str, pd.DataFrame],
                       column_power : str, column_failed : str,
                       title_plot : str) :
    ''' 
    This function generates a plot that compares (1) the cluster power consumption with various scoring plugins and (2)
    the number of pods that fail to be scheduled, as the number of submitted pods increases.
    '''
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    for k, v in competitors_pwr.items() : 
        ax1.plot(v.index, v[column_power], label=k)
    ax1.set_xlabel('Submitted pods')
    ax1.set_ylabel('Watts')
    
    ax2 = ax1.twinx()
    for k, v in competitors_failed.items() : 
        ax2.plot(v.index, v[column_failed], label=f"Cumulative failed pods w/ {k}", linestyle='dashed')
    ax2.set_ylabel("Cumulative failed pods")
    ax2.tick_params(axis='y')
    
    # Legends
    ax1.legend(loc='upper left', bbox_to_anchor=(1.1, 1), borderaxespad=0.)
    ax2.legend(loc='upper left', bbox_to_anchor=(1.1, 0.8), borderaxespad=0.)
    
    plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_energy_savings(competitors_pwr: dict[str, pd.DataFrame], 
                        reference_competitor : str, column_power : str, title_plot : str) :
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    reference = competitors_pwr[reference_competitor]
    for k, v in competitors_pwr.items() :
        if k == reference_competitor: continue
        ax1.plot(v.index, (reference[column_power] - v[column_power]) / reference[column_power] * 100, label=k)
    ax1.set_xlabel('Submitted pods')
    ax1.set_ylabel('% power savings')
    
    # Legends
    ax1.legend(loc='upper right')
    
    plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Retrieve power consumption and failed pods data from the experiments

In [ ]:
# Dictionaries where the results will be stored.
df_pwr_dict = {}
df_sched_pod_dict = {}

DATADIR = "./2024_0531"
data = Path(DATADIR)

fileDirs = sorted([x for x in data.iterdir() if x.is_dir()])
for fdir in fileDirs:
    policyDirs = sorted([x for x in fdir.iterdir() if x.is_dir()])
    for pdir in policyDirs:            
        tuneDirs = sorted([x for x in pdir.iterdir() if x.is_dir()])
        for tdir in tuneDirs:
            seedDirs = sorted([x for x in tdir.iterdir() if x.is_dir()])
            for sdir in seedDirs:
                pwrfile = sdir / 'analysis_pwr.csv'
                schedfile = sdir / 'analysis_cdol.csv'

                
                # Collect telemetry about power consumption.
                try:
                    df_pwr = pd.read_csv(pwrfile)
                    df_pwr.rename(columns = lambda x: x.split('-')[-1], inplace=True)
                    df_pwr_dict.setdefault(pdir.name, list()).append(df_pwr)

                except Exception as e:
                    exit("ERROR: file %s\n%s" % (pwrfile, e))

                
                # Collect telemetry about pods that the cluster failed to schedule.
                try:
                    df_sched_pod = pd.read_csv(schedfile)
                    df_sched_pod.rename(columns = lambda x: x.split('-')[-1], inplace=True)
                    df_sched_pod = df_sched_pod[['event']]
                    df_sched_pod['event'] = 1 * (df_sched_pod['event'] == 'failed')
                    df_sched_pod['event'] = df_sched_pod['event'].cumsum()
                    df_sched_pod_dict.setdefault(pdir.name, list()).append(df_sched_pod)
                
                except Exception as e:
                    exit("ERROR: file %s\n%s" % (schedfile, e))


# display(df_pwr_dict.keys())
# display(df_pwr_dict)
# display(df_sched_pod_dict.keys())
# display(df_sched_pod_dict)

#### Compute the average power consumption and number of failed plugins within each score plugin's set of runs.

In [ ]:
# Compute the average power consumption for each score plugin.
dict_pwr_final_res = {}
for k, v in df_pwr_dict.items() :
    dict_pwr_final_res[k] = sum(v) / len(v)

# Compute the average failed pod for each score plugin.
dict_sched_final_res = {}
for k, v in df_sched_pod_dict.items() :
    dict_sched_final_res[k] = sum(v) / len(v)

    
# display(dict_pwr_final_res.keys())
# display(dict_pwr_final_res)
# display(dict_sched_final_res.keys())
# display(dict_sched_final_res)

### Generation of power consumption plots: overall, GPU only, and CPU only 

In [ ]:
reference_competitor = '06-FGD'

In [ ]:
plot_energy_failed(dict_pwr_final_res, dict_sched_final_res, "power_cluster", "event",
                   'Overall cluster energy consumption vs # submitted pods')
plot_energy_savings(dict_pwr_final_res, reference_competitor, "power_cluster", f"Overall cluster power savings vs {reference_competitor}")

In [ ]:
plot_energy_failed(dict_pwr_final_res, dict_sched_final_res, "power_cluster_GPU", "event",
                   'GPU cluster energy consumption vs # submitted pods')
plot_energy_savings(dict_pwr_final_res, reference_competitor, "power_cluster_GPU", f"GPU cluster power savings vs {reference_competitor}")

In [ ]:
plot_energy_failed(dict_pwr_final_res, dict_sched_final_res, "power_cluster_CPU", "event",
                   'CPU cluster energy consumption vs # submitted pods')
reference_competitor = '06-FGD'
plot_energy_savings(dict_pwr_final_res, reference_competitor, "power_cluster_CPU", f"CPU cluster power savings vs {reference_competitor}")